This is the old version of ***Current Constructing Cloud Perimeter (with tensors).ipynb*** which was extensivley error checked. This notebook performs the same functionality as ***Current Constructing Cloud Perimeter (with tensors).ipynb*** but uses numpy and was hence too slow to be run on all 129 files.

In [ ]:
from netCDF4 import Dataset
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import csv
from datetime import datetime
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import PIL
import warnings
from csv import writer
from scipy import ndimage as nd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
#working perimeter computer (with corrected padding)
def file_generator_padding():
      #initialise a counter for the number of files considered
      counter=0
      #store the path to the COGS files as a variable
      directory = "/content/gdrive/MyDrive/Colab Notebooks/Data/Bath/COGS/"
      #store the file path for the list of all COGS files in chronological order
      file_list_csv = "/content/gdrive/MyDrive/Colab Notebooks/Data/processing_file_list_after23.csv"
      #use pandas to read in the file list csv file and iterate through them
      df = pd.read_csv(file_list_csv)
      for row_index, filename in df.iterrows():
        #initialise a counter which counts the number of times a missing value occurs in a 
        #cell and that cell is therefore discounted  
        missing_value_counter=0
        #initialise the vectors which will store our variables cloud perimeter, cloud fraction
        #and height, these vectors will be the response and explanatory variables for the cells,
        #respectively
        cloud_perimeter_vector=[]
        cloud_fraction_vector=[]
        height_vector=[]
        perimeter_values_vector=[]
        #initialise the vector which stores the dimensions of the datasets
        dimensions=[]
        #print each file name as an error check and to monitor progress of the run
        print(filename['File_names'])
        #access each file name in the list using the key 'File_names'
        filename_str=str(filename['File_names'])
        #store the path to the current file as a variable
        file_path=directory+filename_str[1:-2]
        #add one to the counter as a new file has been accessed
        counter+=1
        #print the file path to check it has been formatted correctly
        print(file_path)
        #read in the netCDF data as a Dataset
        cloud_data_set = Dataset(file_path, mode='r')
        #suppress the warnings that arise when casting as a numpy array
        warnings.filterwarnings("ignore", category=DeprecationWarning) 
        #store the different cloud variables as numpy arrays
        x = np.array(cloud_data_set.variables['x'])
        y = np.array(cloud_data_set.variables['y'])
        z = np.array(cloud_data_set.variables['z'])
        cloud = np.array(cloud_data_set.variables['cloud'])
        time = np.array(cloud_data_set.variables['time'])
        #turn deprecation warnings back on
        warnings.filterwarnings("default", category=DeprecationWarning)
        #append the dimensions of the dataset i.e. the total possible cloud 
        #perimeter values to the end of the dimensions vector
        possible_values=np.shape(cloud)[0]*20*4*4
        #run through all time step in the file
        for t in range(cloud.shape[0]):
          #x and y dimensions have length 121, so only consider the final 120
          #values to make the cloud domain a cube
          cloud_cube=cloud
          cloud_slice=cloud_cube[t,:,:,:]
          perimeter_values_counter=0
          #specify the kernel which allows the computation of the cloud perimeter
          k=[[[0,0,0],[0,1,0],[0,0,0]],[[0,1,0],[1,0,1],[0,1,0]],[[0,0,0],[0,1,0],[0,0,0]]]
          #cast the cloud array and kernels as Tensorflow tensors
          cloud_tensor = tf.constant(cloud_slice, tf.float32)
          k_tensor = tf.constant(k, tf.float32)
          #perform the convolution operation to find an array showing the number of neighbouring 1s for each element
          kernel_cloud = tf.nn.convolution(tf.reshape(cloud_tensor, [1, 120, 121, 121, 1]), tf.reshape(k_tensor, [3, 3, 3, 1, 1]), padding='VALID')
          #subtract this array from 6 in every entry to find the perimeter of each element
          kernel_cloud_subtracted = tf.subtract(6,kernel_cloud)
          #remove the outer edges of the cloud array so that we can multiply by the convolved array entry-wise
          #edges are removed because VALID padding is used
          cloud_tensor_reduced=cloud_tensor[1:-1,1:-1,1:-1]
          #multiply the two arrays entry-wise so that we only consider the perimeter of elements where a cloud i.e. a 1
          #exists (where a 0 exists, this entry will just be zeroed by the pointwise operation)
          cloud_perimeter_tensor=tf.math.multiply(cloud_tensor_reduced,tf.squeeze(kernel_cloud_subtracted))
          #loop over all 320 cells in the domain
          for kk in range(20):
              for jj in range(4):
                  for ii in range(4):
                      if (kk==19 and jj==3 and ii==3):
                          #when we lie in the final cell we overlap by as much as necessary
                          cloud_cell=cloud_tensor_reduced[-6:,-30:,-30:]
                          cloud_perimeter_cell=cloud_perimeter_tensor[-6:,-30:,-30:]
                      elif (kk==19 and jj!=3 and ii!=3):
                          #when we lie in the final cell we overlap by as much as necessary
                          cloud_cell=cloud_tensor_reduced[-6:,30*(jj):30*(jj+1),30*(ii):30*(ii+1)]
                          cloud_perimeter_cell=cloud_perimeter_tensor[-6:,30*(jj):30*(jj+1),30*(ii):30*(ii+1)]
                      elif (kk==19 and jj==3 and ii!=3):
                          #when we lie in the final cell we overlap by as much as necessary
                          cloud_cell=cloud_tensor_reduced[-6:,-30:,30*(ii):30*(ii+1)]
                          cloud_perimeter_cell=cloud_perimeter_tensor[-6:,-30:,30*(ii):30*(ii+1)]
                      elif (kk==19 and jj!=3 and ii==3):
                          #when we lie in the final cell we overlap by as much as necessary
                          cloud_cell=cloud_tensor_reduced[-6:,30*(jj):30*(jj+1),-30:]
                          cloud_perimeter_cell=cloud_perimeter_tensor[-6:,30*(jj):30*(jj+1),-30:]
                      elif (kk!=19 and jj!=3 and ii==3):
                          #when we lie in the final cell we overlap by as much as necessary
                          cloud_cell=cloud_tensor_reduced[6*kk:6*(kk+1),30*(jj):30*(jj+1),-30:]
                          cloud_perimeter_cell=cloud_perimeter_tensor[6*kk:6*(kk+1),30*(jj):30*(jj+1),-30:]
                      elif (kk!=19 and jj==3 and ii==3):
                          #when we lie in the final cell we overlap by as much as necessary
                          cloud_cell=cloud_tensor_reduced[6*kk:6*(kk+1),-30:,-30:]
                          cloud_perimeter_cell=cloud_perimeter_tensor[6*kk:6*(kk+1),-30:,-30:]
                      elif (kk!=19 and jj==3 and ii!=3):
                          #when we lie in the final cell we overlap by as much as necessary
                          cloud_cell=cloud_tensor_reduced[6*kk:6*(kk+1),-30:,30*(ii):30*(ii+1)]
                          cloud_perimeter_cell=cloud_perimeter_tensor[6*kk:6*(kk+1),-30:,30*(ii):30*(ii+1)]
                      else:
                          #specify which of the interior 320 cells that we lie in
                          cloud_cell=cloud_tensor_reduced[6*kk:6*(kk+1),30*(jj):30*(jj+1),30*(ii):30*(ii+1)]
                          #specify the equivalent cell for the cloud_perimeter_tensor generated above
                          cloud_perimeter_cell=cloud_perimeter_tensor[6*kk:6*(kk+1),30*(jj):30*(jj+1),30*(ii):30*(ii+1)]
                      #if any value within the cell is -1 we discount the cell entirely,
                      #as prescribed by the methodology
                      if np.any(cloud_cell < 0):
                          #increment the counter by 1 if we ignore a cell
                          missing_value_counter+=1
                          continue
                      else:
                          perimeter_values_counter+=1
                          #compute the cloud fraction - divide the number of 1s by the volume of a cell
                          cloud_fraction_value=(np.sum(cloud_cell))/(np.shape(cloud_cell)[0]*np.shape(cloud_cell)[1]*np.shape(cloud_cell)[2])
                          cloud_fraction_value=float(cloud_fraction_value)
                          #append the computed value onto the cloud fraction vector 
                          cloud_fraction_vector.append(cloud_fraction_value)
                          #sum over the entire 3D array to find the total perimeter - a single scalar
                          cloud_perimeter_value=tf.reduce_sum(cloud_perimeter_cell)
                          cloud_perimeter_value=int(cloud_perimeter_value)
                          #normalise the cloud perimeter value by dividing it by the maximum perimeter for a cell (16200)
                          cloud_perimeter_value=float(cloud_perimeter_value/16200)
                          #append the cloud perimeter value for this time step in this cell to vector
                          cloud_perimeter_vector.append(cloud_perimeter_value)

                          #append the height to the height vector
                          height_vector.append(kk)
          #for each time step append the number of cells in which a perimeter value was
          #recorded (this will be used in the sonde data function below)                
          perimeter_values_vector.append(perimeter_values_counter)

        #write the lists to numpy arrays
        cloud_perimeter_vector=np.array(cloud_perimeter_vector)
        cloud_fraction_vector=np.array(cloud_fraction_vector)
        height_vector=np.array(height_vector)
        perimeter_values_vector=np.array(perimeter_values_vector)
        #print lines to check that the total number of cells for which a perimeter value was recorded
        #plus the number of cells discounted due to a missing value equals the total possible cells
        print("Sum of cloud perimeter cells=",np.sum(perimeter_values_vector))
        print("Missing value cells=",missing_value_counter)
        print("Total possible cells=",cloud.shape[0]*20*4*4)
        #save the arrays to files on google drive labelled by the name of the COGS file
        new_file_path_perimeter="/content/gdrive/MyDrive/Colab Notebooks/Data/Variables/Cloud Perimeter/"
        new_file_path_fraction="/content/gdrive/MyDrive/Colab Notebooks/Data/Variables/Cloud Fraction/"
        new_file_path_height="/content/gdrive/MyDrive/Colab Notebooks/Data/Variables/Height/"
        new_file_path_counter="/content/gdrive/MyDrive/Colab Notebooks/Data/Variables/Counter/"
        new_file_name_perimeter=new_file_path_perimeter+"Perimeter_"+filename_str[1:-5]+".csv"
        new_file_name_fraction=new_file_path_fraction+"Fraction_"+filename_str[1:-5]+".csv"
        new_file_name_height=new_file_path_height+"Height_"+filename_str[1:-5]+".csv"
        new_file_name_counter=new_file_path_counter+"Counter_"+filename_str[1:-5]+".csv"
        np.savetxt(new_file_name_perimeter, cloud_perimeter_vector)
        np.savetxt(new_file_name_fraction, cloud_fraction_vector)
        np.savetxt(new_file_name_height, height_vector)
        np.savetxt(new_file_name_counter, perimeter_values_vector)


        df=df.drop(index=row_index)
        df.to_csv(file_list_csv, index=False)



        print(new_file_name_perimeter)
        print('Counter='+str(counter))
        yield counter

In [ ]:
all_files_iterator=True
for i in file_generator_padding():
  if i>200:
    break

'sgpcogspiN1.c1.20180531.141920.nc',
/content/gdrive/MyDrive/Colab Notebooks/Data/Bath/COGS/sgpcogspiN1.c1.20180531.141920.nc


KeyboardInterrupt: ignored

In [ ]:
def fill(data, invalid=None):
    """
    Replace the value of invalid 'data' cells (indicated by 'invalid') 
    by the value of the nearest valid data cell

    Input:
        data:    numpy array of any dimension
        invalid: a binary array of same shape as 'data'. True cells set where data
                 value should be replaced.
                 If None (default), use: invalid  = np.isnan(data)

    Output: 
        Return a filled array. 
    """

    if invalid is None: invalid = np.isnan(data)

    ind = nd.distance_transform_edt(invalid, return_distances=False, return_indices=True)
    return data[tuple(ind)]

In [ ]:
# working atmospheric alignment
def atmospheric_generator_simple():
    # initialise a counter for the number of files considered
    counter = 0
    # store the path to the COGS files as a variable
    directory = "/content/gdrive/MyDrive/Colab Notebooks/Data/Bath/COGS/"
    # specify the folder containing the atmospheric files
    atmospheric_directory = "/content/gdrive/MyDrive/Colab Notebooks/Data/Bath/"
    # specify the folder containing the cloud perimeter value counter files
    counter_directory = "/content/gdrive/MyDrive/Colab Notebooks/Data/Variables/Counter/"
    # store the file path for the list of all COGS files in chronological order
    file_list_csv = "/content/gdrive/MyDrive/Colab Notebooks/Data/processing_file_list_after23.csv"
    # use pandas to read in the file list csv file and iterate through them
    df = pd.read_csv(file_list_csv)
    for row_index, filename in df.iterrows():
        print(filename['File_names'])
        # access each file name in the list using the key 'File_names'
        filename_str = str(filename['File_names'])
        # print the date as an error check
        print(filename_str[16:-12])
        # find the name of the corresponding atmospheric file
        atmospheric_filename = 'sgpinterpolatedsondeC1.c1.' + \
            filename_str[16:-12]+'.000030.nc'
        # specify the path to the atmospheric file
        atmospheric_file_path = atmospheric_directory + atmospheric_filename
        # print the file path to check it has been formatted correctly
        print(atmospheric_file_path)
        # read in the netCDF data as a Dataset
        interpolated_data = Dataset(atmospheric_file_path, mode='r')
        # store the path to the current cloud file as a variable
        file_path = directory+filename_str[1:-2]
        # read in the netCDF data as a Dataset
        cloud_data_set = Dataset(file_path, mode='r')
        # access the counter file from the directory and store as a dataframe
        perimeter_counter_file_path = counter_directory + \
            "Counter_"+filename_str[1:-5]+".csv"
        counter_df = pd.read_csv(perimeter_counter_file_path, header=None)
        # then convert to numpy array of integers
        counter_df = counter_df.to_numpy(dtype='i')
        # add one to the counter as all new files have been accessed
        counter += 1
        # suppress the warnings that arise when casting as a numpy array
        warnings.filterwarnings("ignore", category=DeprecationWarning)
        # store the different cloud variables as numpy arrays
        #x = np.array(cloud_data_set.variables['x'])
        #y = np.array(cloud_data_set.variables['y'])
        #z = np.array(cloud_data_set.variables['z'])
        #cloud = np.array(cloud_data_set.variables['cloud'])
        cloud_time = np.array(cloud_data_set.variables['time'])
        # cast the net cdf atmospheric variables as numpy arrays
        base_time = np.array(interpolated_data.variables['base_time'])
        time_offset = np.array(interpolated_data.variables['time_offset'])
        interpolated_time = np.array(interpolated_data.variables['time'])
        height = np.array(interpolated_data.variables['height'])
        u_wind = np.array(interpolated_data.variables['u_wind'])
        qc_u_wind = np.array(interpolated_data.variables['qc_u_wind'])
        v_wind = np.array(interpolated_data.variables['v_wind'])
        qc_v_wind = np.array(interpolated_data.variables['qc_v_wind'])
        precip = np.array(interpolated_data.variables['precip'])
        qc_precip = np.array(interpolated_data.variables['qc_precip'])
        temp = np.array(interpolated_data.variables['temp'])
        qc_temp = np.array(interpolated_data.variables['qc_temp'])
        bar_pres = np.array(interpolated_data.variables['bar_pres'])
        qc_bar_pres = np.array(interpolated_data.variables['qc_bar_pres'])
        rh_scaled = np.array(interpolated_data.variables['rh_scaled'])
        qc_rh_scaled = np.array(interpolated_data.variables['qc_rh_scaled'])
        potential_temp = np.array(
            interpolated_data.variables['potential_temp'])
        qc_potential_temp = np.array(
            interpolated_data.variables['qc_potential_temp'])
        sh = np.array(interpolated_data.variables['sh'])
        qc_sh = np.array(interpolated_data.variables['qc_sh'])
        wdir = np.array(interpolated_data.variables['wdir'])
        qc_wdir = np.array(interpolated_data.variables['qc_wdir'])
        vap_pres = np.array(interpolated_data.variables['vap_pres'])
        qc_vap_pres = np.array(interpolated_data.variables['qc_vap_pres'])
        dp = np.array(interpolated_data.variables['dp'])
        qc_dp = np.array(interpolated_data.variables['qc_dp'])
        wspd = np.array(interpolated_data.variables['wspd'])
        qc_wspd = np.array(interpolated_data.variables['qc_wspd'])
        # turn deprecation warnings back on
        warnings.filterwarnings("default", category=DeprecationWarning)

        '''#replace all values which are not quality with the mean of all quality readings
    #THIS MAY NOT BE THE BEST STRATEGY BUT SHOULD ESSENTIALLY TELL THE NEURAL NET
    #TO DO THE USUAL THING WITH THAT VARIABLE
    wspd[qc_wspd>=1]=np.mean(wspd[qc_wspd<1])
    u_wind[qc_u_wind>=1]=np.mean(u_wind[qc_u_wind<1])
    v_wind[qc_v_wind>=1]=np.mean(v_wind[qc_v_wind<1])
    temp[qc_temp>=1]=np.mean(temp[qc_temp<1])
    bar_pres[qc_bar_pres>=1]=np.mean(bar_pres[qc_bar_pres<1])
    rh_scaled[qc_rh_scaled>=1]=np.mean(rh_scaled[qc_rh_scaled<1])
    potential_temp[qc_potential_temp>=1]=np.mean(potential_temp[qc_potential_temp<1])
    sh[qc_sh>=1]=np.mean(sh[qc_sh<1])
    wdir[qc_wdir>=1]=np.mean(wdir[qc_wdir<1])
    vap_pres[qc_vap_pres>=1]=np.mean(vap_pres[qc_vap_pres<1])
    dp[qc_dp>=1]=np.mean(dp[qc_dp<1])'''

        #instead use the fill function defined in the previous cell to replace
        #any invalid value with the nearest valid value
        wspd_invalid= qc_wspd>=1
        u_wind_invalid= qc_u_wind>=1
        v_wind_invalid= qc_v_wind>=1
        temp_invalid= qc_temp>=1
        bar_pres_invalid= qc_bar_pres>=1
        rh_scaled_invalid= qc_rh_scaled>=1
        potential_temp_invalid= qc_potential_temp>=1
        sh_invalid= qc_sh>=1
        vap_pres_invalid= qc_vap_pres>=1
        dp_invalid= qc_dp>=1

        wspd = fill(wspd, invalid=wspd_invalid)
        u_wind = fill(u_wind, invalid=u_wind_invalid)
        v_wind = fill(v_wind, invalid=v_wind_invalid)
        temp = fill(temp, invalid=temp_invalid)
        bar_pres = fill(bar_pres, invalid=bar_pres_invalid)
        rh_scaled = fill(rh_scaled, invalid=rh_scaled_invalid)
        potential_temp = fill(potential_temp, invalid=potential_temp_invalid)
        sh = fill(sh, invalid=sh_invalid)
        vap_pres = fill(vap_pres, invalid=vap_pres_invalid)
        dp = fill(dp, invalid=dp_invalid)

        # normalise the variables ready for the neural network
        rh_scaled = (rh_scaled - np.mean(rh_scaled)) / np.std(rh_scaled)
        wspd = (wspd - np.mean(wspd)) / np.std(wspd)
        u_wind = (u_wind - np.mean(u_wind)) / np.std(u_wind)
        v_wind = (v_wind - np.mean(v_wind)) / np.std(v_wind)
        temp = (temp - np.mean(temp)) / np.std(temp)
        bar_pres = (bar_pres - np.mean(bar_pres)) / np.std(bar_pres)
        potential_temp = (potential_temp - np.mean(potential_temp)) / np.std(potential_temp)
        sh = (sh - np.mean(sh)) / np.std(sh)
        vap_pres = (vap_pres - np.mean(vap_pres)) / np.std(vap_pres)
        dp = (dp - np.mean(dp)) / np.std(dp)

        # find the height dimension
        height_dimension = np.shape(u_wind)[1]
        atmospheric_variables_array = np.zeros((2,int(height_dimension),10))

        for i in range(len(cloud_time)):
            assert len(cloud_time) == len(
                counter_df), "mismatch between number of time steps in cloud_time and timesteps in the number of perimeter values vector"
            num_perimeter_values = int(counter_df[i])
            index = (cloud_time[i] - int(base_time)) // 60
            if (cloud_time[i] % 60) == 0:
                index = index - 1
            # initialise an np array
            atmospheric_variables_mini_array = np.zeros((2,int(height_dimension),10))
            atmospheric_variables = np.zeros((int(height_dimension),10))
            atmospheric_variables[:, 0] = wspd[index, :]
            atmospheric_variables[:, 1] = u_wind[index, :]
            atmospheric_variables[:, 2] = v_wind[index, :]
            atmospheric_variables[:, 3] = temp[index, :]
            atmospheric_variables[:, 4] = bar_pres[index, :]
            atmospheric_variables[:, 5] = rh_scaled[index, :]
            atmospheric_variables[:, 6,] = potential_temp[index, :]
            atmospheric_variables[:, 7] = sh[index, :]
            atmospheric_variables[:, 8] = vap_pres[index, :]
            atmospheric_variables[:, 9] = dp[index, :]
            for k in range(num_perimeter_values):
                atmospheric_variables_mini_array = np.concatenate((atmospheric_variables_mini_array,
                         atmospheric_variables.reshape(1,atmospheric_variables.shape[0],atmospheric_variables.shape[1])), axis=0)
            atmospheric_variables_mini_array = atmospheric_variables_mini_array[2:, :, :]
            atmospheric_variables_array = np.concatenate((atmospheric_variables_array,
                         atmospheric_variables_mini_array), axis=0)
        atmospheric_variables_array = atmospheric_variables_array[2:, :, :]


        new_file_path_atmospheric = "/content/gdrive/MyDrive/Colab Notebooks/Data/Variables/Atmospheric/"
        new_file_name_atmospheric = new_file_path_atmospheric + \
            "Atmospheric_"+filename_str[1:-5]+".npy"
        print(atmospheric_variables_array.shape)
        print(int(len(cloud_time))*num_perimeter_values)

        #alternative save to csv
        '''#reshape the array from a 3D matrix to a 2D matrix
        atmospheric_variables_array_reshaped = atmospheric_variables_array.reshape(atmospheric_variables_array.shape[0], -1)

        #save reshaped array to file
        np.savetxt(new_file_name_atmospheric, atmospheric_variables_array_reshaped)
        
        #error check
        if (index==0 or index==1):
              #retrieving data from file.
              loaded_arr = np.loadtxt(new_file_name_atmospheric)

              #The loaded array is a 2D array, therefore we need to convert it to the original
              #array shape to get original matrix with original shape.
              load_original_arr = loaded_arr.reshape(loaded_arr.shape[0], loaded_arr.shape[1] // atmospheric_variables_array.shape[2], atmospheric_variables_array.shape[2])

              # check the shapes:
              print("shape of arr: ", atmospheric_variables_array.shape)
              print("shape of load_original_arr: ", load_original_arr.shape)

              # check if both arrays are same or not:
              if (load_original_arr == atmospheric_variables_array).all():
                print("Yes, both the arrays are same")
              else:
                print("No, both the arrays are not same")
        '''


        atmospheric_variables_array = atmospheric_variables_array.astype(np.float16)
        np.save(new_file_name_atmospheric, atmospheric_variables_array)

        df=df.drop(index=row_index)
        df.to_csv(file_list_csv)

        print(new_file_name_atmospheric)
        print('Counter='+str(counter))
        yield counter

In [ ]:
atmospheric_files_iterator=True
for i in atmospheric_generator_simple():
  if i>200:
    break

'sgpcogspiN1.c1.20180704.190000.nc',
20180704
/content/gdrive/MyDrive/Colab Notebooks/Data/Bath/sgpinterpolatedsondeC1.c1.20180704.000030.nc


KeyboardInterrupt: ignored